# notebook to prepare data

In [1]:
import os
import shutil
import pandas as pd
import numpy as np

In [2]:
root_path = '/mnt/c/Arbeid/Github_Repo/MSAD_work/'

In [3]:
def concat_results(directory):
    dfs = list()
    
    # Iterate through execution dates
    for exec_folder in os.listdir(directory):
        for file in os.listdir(os.path.join(directory, exec_folder)):
            if file == 'results.csv':
                filepath = os.path.join(directory, exec_folder, file)
                df = pd.read_csv(filepath)
                df['result_path'] = filepath
                dfs.append(df)
    
    return pd.concat([df.reset_index(drop=True) for df in dfs], ignore_index=True)

directory = '/mnt/c/Arbeid/Github_Repo/TimeEval_work/results'
all_results = concat_results(directory)

In [4]:
obsea_results = all_results[(all_results['collection'] == 'OBSEA') | (all_results['collection'] == 'OBSEA_2')]
obsea_results = obsea_results[(obsea_results['BestF1Score'].notna()) & (obsea_results['FalseNegativeRate'].notna())]
obsea_results['collection'][obsea_results['collection'] == 'OBSEA_2'] = 'OBSEA'
obsea_results = obsea_results.groupby(['algorithm', 'collection', 'dataset']).max().reset_index()
obsea_results['Recommendation_ACC'] =  obsea_results.apply(lambda row: max(row['BestF1Score'] - row['FalseNegativeRate'], 0), axis=1)
obsea_results

,algorithm,collection,dataset,algo_training_type,algo_input_dimensionality,dataset_training_type,dataset_input_dimensionality,status,hyper_params_id,repetition,...,execute_postprocess_time,ROC_AUC,RANGE_PR_AUC,BestF1Score,FalseNegativeRate,result_path,train_main_time,train_preprocess_time,train_postprocess_time,Recommendation_ACC
0,AutoEncoder (AE),OBSEA,2020-01-01,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,dfdc9a9fdc00b09d9d188aaabfa8f91c,3,...,NaN,NaN,NaN,0.000000,0.000000,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,35.058387,NaN,NaN,0.000000
1,AutoEncoder (AE),OBSEA,2020-01-02,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,dfdc9a9fdc00b09d9d188aaabfa8f91c,3,...,NaN,0.136121,0.190908,0.023367,0.010989,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,27.505230,NaN,NaN,0.012378
2,AutoEncoder (AE),OBSEA,2020-01-03,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,dfdc9a9fdc00b09d9d188aaabfa8f91c,3,...,NaN,0.902499,0.297292,0.336366,0.017730,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,25.895189,NaN,NaN,0.318636
3,AutoEncoder (AE),OBSEA,2020-01-04,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,dfdc9a9fdc00b09d9d188aaabfa8f91c,3,...,NaN,0.903116,0.438966,0.876110,0.132974,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,24.888561,NaN,NaN,0.743136
4,AutoEncoder (AE),OBSEA,2020-01-05,semi-supervised,multivariate,multivariate,semi-supervised,Status.OK,dfdc9a9fdc00b09d9d188aaabfa8f91c,3,...,NaN,0.609201,0.134994,0.107135,0.010909,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,24.670873,NaN,NaN,0.096226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,Torsk,OBSEA,2020-12-17_unsupervised,unsupervised,multivariate,multivariate,unsupervised,Status.OK,de9b8b973f50539dc5ef4c8c1a722f4b,10,...,0.025198,0.748521,0.807776,0.790718,0.377759,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,NaN,NaN,NaN,0.412959
2860,Torsk,OBSEA,2020-12-18_unsupervised,unsupervised,multivariate,multivariate,unsupervised,Status.OK,de9b8b973f50539dc5ef4c8c1a722f4b,10,...,0.017805,0.749227,0.879835,0.803628,0.111496,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,NaN,NaN,NaN,0.692132
2861,Torsk,OBSEA,2020-12-19_unsupervised,unsupervised,multivariate,multivariate,unsupervised,Status.OK,de9b8b973f50539dc5ef4c8c1a722f4b,10,...,0.020071,0.714787,0.807692,0.719457,0.470284,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,NaN,NaN,NaN,0.249173
2862,Torsk,OBSEA,2020-12-20_unsupervised,unsupervised,multivariate,multivariate,unsupervised,Status.OK,de9b8b973f50539dc5ef4c8c1a722f4b,10,...,0.021138,0.681948,0.661543,0.641608,0.474343,/mnt/c/Arbeid/Github_Repo/TimeEval_work/result...,NaN,NaN,NaN,0.167265


In [5]:
#obsea_results = obsea_results[:500]
#obsea_results

In [6]:
obsea_results['algorithm'].unique()

array(['AutoEncoder (AE)', 'CBLOF', 'COPOD', 'DeepAnT',
       'DenoisingAutoEncoder (DAE)', 'EncDec-AD', 'HBOS', 'Hybrid KNN',
       'LOF', 'PCC', 'Random Black Forest (RR)', 'RobustPCA', 'Torsk'],
      dtype=object)

In [7]:
# copy data
dest_path = root_path + 'data/OBSEA/data'
src_path = '/mnt/c/Arbeid/Github_Repo/TimeEval_work/work_data/processed/multivariate/OBSEA'
for data_file in os.listdir(src_path):
    if 'test' in data_file:
        df = pd.read_csv(src_path + '/' + data_file)
        df.drop(df.columns[0], axis=1, inplace=True)
        df.to_csv(dest_path + '/' + data_file.split('.')[0] + '.out', index=False, header=False)

In [11]:
# copy scores
dest_path = root_path + 'data/OBSEA/scores'
for row_idx in range(len(obsea_results)):
    try:
        result_path = '/'.join(obsea_results.iloc[row_idx, obsea_results.columns.get_loc('result_path')].split('/')[:-1])
        algorithm = obsea_results.iloc[row_idx, obsea_results.columns.get_loc('algorithm')]
        hyper_params_id = obsea_results.iloc[row_idx, obsea_results.columns.get_loc('hyper_params_id')]
        collection = obsea_results.iloc[row_idx, obsea_results.columns.get_loc('collection')]
        dataset = obsea_results.iloc[row_idx, obsea_results.columns.get_loc('dataset')]
        repetition = str(obsea_results.iloc[row_idx, obsea_results.columns.get_loc('repetition')])
        scores_file_name = 'docker-algorithm-scores.csv'
        src_path = result_path + '/' + algorithm + '/' + hyper_params_id + '/' + collection + '/' + dataset + '/' + repetition + '/' + scores_file_name
        scores_dir_alg = dest_path + '/' + algorithm
        os.makedirs(scores_dir_alg, exist_ok=True)
        dest_score_path = scores_dir_alg + '/' + dataset.split('_')[0] + '.out'
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_score_path)
        else:
            src_path = src_path.replace('OBSEA', 'OBSEA_2')
            shutil.copy(src_path, dest_score_path)
    except:
        pass

In [8]:
# copy metrics
path_to_metrics = root_path + 'data/OBSEA/metrics'
metrics = ['ROC_AUC', 'RANGE_PR_AUC', 'BestF1Score', 'FalseNegativeRate', 'Recommendation_ACC']
exec_algorithms = obsea_results['algorithm'].unique()
for exec_algorithm in exec_algorithms:
    for metric in metrics:
        df = obsea_results[(obsea_results['algorithm'] == exec_algorithm)]
        df = df[['dataset', metric]]
        df['dataset'] = df['dataset'].str.strip().str.replace('_unsupervised', '.out')
        df['dataset'] = df['dataset'].apply(lambda x: x + '.out' if not x.endswith('.out') else x)
        df['dataset'] = df['dataset'].apply(lambda x: 'OBSEA/' + x)
        df.set_index('dataset', inplace=True)
        df.index.name = ''
        df = df.groupby(level=0).min()
        df.rename(columns={metric: exec_algorithm}, inplace=True)
        df.dropna(inplace=True)
        metrics_dir_alg = path_to_metrics + '/' + exec_algorithm
        os.makedirs(metrics_dir_alg, exist_ok=True)
        #print(df)
        df.to_csv(metrics_dir_alg + '/' + metric + '.csv')